In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from conllu import TokenList
from mlflow.models import infer_signature
from mlflow import xgboost
import polars
from polars import col
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from label_legends.preprocess import create_conllu, holdout, load_conllu, load_data, load_test, load_train, tfidf_pipeline, token_lists, transform, load_vectorizer, reverse_vocabulary, vocabulary, ids_to_tokens, tokens_to_ids, vectorize_tokens, strip_stopwords, load_test
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import logging
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, precision_recall_curve
import mlflow

np.set_printoptions(threshold=1000)
logging.basicConfig(level=logging.INFO)

In [16]:
load_data().collect()

id,rewire_id,text,annotator,label_sexist,label_category,label_vector,split,tokens
i64,str,str,i64,str,str,str,str,list[str]
0,"""sexism2022_english-0""",""" I wonder what keeps that witc…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train""","[""i"", ""wonder"", … ""😄""]"
1,"""sexism2022_english-0""",""" I wonder what keeps that witc…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train""","[""i"", ""wonder"", … ""😄""]"
10,"""sexism2022_english-100""","""Good for her! My grandson had …",3,"""not sexist""","""none""","""none""","""train""","[""good"", ""for"", … ""!""]"
100,"""sexism2022_english-10026""","""It is not insulting, it's **ex…",14,"""sexist""","""2. derogation""","""2.1 descriptive attacks""","""test""","[""it"", ""be"", … "".**""]"
1000,"""sexism2022_english-10297""","""The article said Madonna offer…",5,"""sexist""","""2. derogation""","""2.3 dehumanising attacks & ove…","""train""","[""the"", ""article"", … "".""]"
…,…,…,…,…,…,…,…,…
9995,"""sexism2022_english-12996""","""Shudder.. if you had to have s…",17,"""sexist""","""2. derogation""","""2.3 dehumanising attacks & ove…","""test""","[""shudder"", "".."", … "".""]"
9996,"""sexism2022_english-12997""","""You mean one that forces women…",6,"""not sexist""","""none""","""none""","""train""","[""you"", ""mean"", … ""?""]"
9997,"""sexism2022_english-12997""","""You mean one that forces women…",4,"""not sexist""","""none""","""none""","""train""","[""you"", ""mean"", … ""?""]"


In [17]:
mlflow.autolog()

2024/12/11 17:28:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


2024/12/11 17:28:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [18]:
val, tra = holdout()
tra = transform(tra)
val = transform(val)

test = transform(load_test().collect())

In [19]:
tra

id,text,tokens,token_ids,label
i64,str,list[str],list[i64],i64
26016,"""#VoteDemOut seditionist wanna …","[""#"", ""votedemout"", … ""ven""]","[7, 3000, … 3000]",0
41060,"""Cerno hops from wave to wave: …","[""cerno"", ""hop"", … "".""]","[3000, 3000, … 26]",1
35766,"""think about it.. he´s called J…","[""think"", "".."", … ""elect""]","[2688, 27, … 911]",0
23678,"""Hmm, you could rewrite this wi…","[""hmm"", "","", … ""justsaying""]","[1284, 22, … 3000]",0
52396,"""Every girl in a game, or on si…","[""girl"", ""game"", … "".""]","[1164, 1135, … 26]",0
…,…,…,…,…
6785,"""I just think my dick couldn't …","[""just"", ""think"", … ""..😂😂😂""]","[1482, 2688, … 3000]",0
31355,"""no he deadasss hit the nail on…","[""deadass"", ""hit"", … "".""]","[3000, 1282, … 26]",0
49834,"""Aren't these goblins always wh…","[""goblins"", ""whine"", … ""hastagsarestillcool""]","[3000, 2914, … 3000]",0


In [20]:
tfidf_pipeline().fit(token_lists(tra))

2024/12/11 17:28:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8a8175676f6c471f865b58cd5320412a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2024/12/11 17:28:05 WARNING mlflow.utils: Truncated the value of the key `count__vocabulary`. Truncated value: `{'wonder': np.int64(2940), 'witch': np.int64(2933), 'look': np.int64(1607), 'quite': np.int64(2152), 'young': np.int64(2975), '70': np.int64(80), '.': np.int64(26), 'maybe': np.int64(1679), 'start': np.int64(2546), 'drink': np.int64(872), 'blood': np.int64(378), '?': np.int64(98), 'good': np.int64(1179), '!': np.int64(0), 'acid': np.int64(132), ',': np.int64(22), 'mother': np.int64(1764), 'like': np.int64(1579), 'yes': np.int64(2971), 'mean': np.int64(1683), 'mute': np.int64(1788), 'article': np.int64(244), 'say': np.int64(2330), 'madonna': np.int64(1636), 'offer': np.int64(1879), 'head': np.int64(1253), 'vote': np.int64(2870), 'hillary': np.int64(1277), 'whore': np.int64(2917), 'mouth': np.int64(1768), 'answer': np.int64(205), '60': np.int64(78), '%': np.int64(11), 'america': np.int64(195), 'ask': np.int64(251), '#': np.int64(7), 'soy': np.int64(2507), 'grow': np.int64(1206)

2024/12/11 17:28:06 WARNING mlflow.sklearn: Unrecognized dataset type <class 'list'>. Dataset logging skipped.


/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


2024/12/11 17:28:06 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///home/lukas/Programming/uni/nlp-ie-label-legends/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`


Pipeline(steps=[('count',
                 CountVectorizer(lowercase=False, max_features=3000,
                                 stop_words='english',
                                 tokenizer=<label_legends.preprocess.ConlluTokenizer object at 0x7e06a66c0470>,
                                 vocabulary={'!': np.int64(0),
                                             '!!': np.int64(1),
                                             '!!!': np.int64(2),
                                             '!!!!': np.int64(3),
                                             '!!!!!': np.int64(4),
                                             '!?': np.int64(5),
                                             '"': np.int64(6), '#': np.int64(7),
                                             '#trump': np.int64(8),
                                             '#x200b...
                                             '&': np.int64(12),
                                             "'": np.int64(13),
                                             "''": np.int64(14),
                                             "'s": np.int64(15),
                                             '(': np.int64(16),
                                             ')': np.int64(17),
                                             '*': np.int64(18),
                                             '**': np.int64(19),
                                             '**.': np.int64(20),
                                             '+': np.int64(21),
                                             ',': np.int64(22),
                                             '-': np.int64(23),
                                             '--': np.int64(24),
                                             '---': np.int64(25),
                                             '.': np.int64(26),
                                             '..': np.int64(27),
                                             '...': np.int64(28),
                                             '....': np.int64(29), ...})),
                ('tfidf', TfidfTransformer())])

In [21]:
X_train = tfidf_pipeline().transform(token_lists(tra))
X_test = tfidf_pipeline().transform(token_lists(test))

In [22]:
clf = XGBClassifier()
clf.fit(X_train, tra.select("label").to_series().to_list())

2024/12/11 17:28:08 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cb34feb36f2b4d90aef7e1568fd564d1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


2024/12/11 17:28:16 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/mlflow/xgboost/__init__.py", line 738, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/mlflow/xgboost/__init__.py", line 637, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/mlflow/tracking/fluent.py", line 1179, in log_artifact
    MlflowClient().log_artifact(run_id, local_path, artifact_path)
  File "/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/mlflow/tracking/client.py", line 1933, in log_artifact
    self._tracking_client.log_artifact(run_id, l

2024/12/11 17:28:16 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///home/lukas/Programming/uni/nlp-ie-label-legends/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`


2024/12/11 17:28:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/lukas/Programming/uni/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:28:16] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


2024/12/11 17:28:19 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2024/12/11 17:28:19 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///home/lukas/Programming/uni/nlp-ie-label-legends/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
y_pred = clf.predict(X_test)
y_true = test.select("label").to_series().to_list()

In [24]:
confusion_matrix(y_true, y_pred)

array([[8511,  374],
       [1905, 1210]])

In [25]:
accuracy_score(y_true, y_pred)

0.8100833333333334

In [26]:
precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred)
print(f"precision: {precision} recall: {recall} fscore: {fscore} support: {support}")

precision: [0.81710829 0.76388889] recall: [0.95790658 0.38844302] fscore: [0.88192322 0.51500319] support: [8885 3115]


In [ ]:
with mlflow.start_run():
    # hyperparams
    mlflow.log_params(params)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.set_tag("XGBoost", "Baseline")
    signature = infer_signature(X_train, y_pred)

    # log the model
    model_info = mlflow.xgboost.log_model(
        model=clf,
        artifact_path